In [ ]:
%pip install boto3 pyspark delta-spark python-dotenv
%pip install pandas

In [ ]:
import os
from pyspark.sql import SparkSession, types
from IPython.display import display
import pandas as pd
import json
import boto3

In [ ]:
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'uZCPe6xoPsMQymYaBwPk')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'iLA2wO10sexC0RzUHodV0xUpTqXPFJnfubXFGJBt')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')

In [ ]:
path_1 = "s3a://data/data-raw/data.json"
path_2 = "s3a://data/data-raw/data2.json"
path_3 = "s3a://data/data-raw/data3.json"

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config('spark.hadoop.fs.s3a.path.style.access', "true") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.access.key", 'uZCPe6xoPsMQymYaBwPk') \
    .config("spark.hadoop.fs.s3a.secret.key", 'iLA2wO10sexC0RzUHodV0xUpTqXPFJnfubXFGJBt') \
    .config("spark.hadoop.fs.s3a.endpoint", 'http://localhost:9000') \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "true") \
    .getOrCreate()

In [ ]:
# Reading each file in the bucket

df1 = spark.read.option("multiline", "true") \
    .json(path_1)
df2 = spark.read.option("multiline", "true") \
    .json(path_2)
df3 = spark.read.option("multiline", "true") \
    .json(path_3)

In [ ]:
# Showing each dataframe after reading

df1.show()
df2.show()
df3.show()

In [ ]:
# The function from the "download.ipynb" to upload the result.json

def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)

In [ ]:
# Merging each dataframe from data.json, data2.json, data3.json

merged_df = df1.union(df2)
merged_df1 = merged_df.union(df3)
merged_df1.show()

In [ ]:
#Changing the PySpark dataframe to Pandas dataframe, the reason was I continously ate error in the face while trying write JSON file by Spark =)

print(type(merged_df1));
result = merged_df1.toPandas()
print(type(result));

In [ ]:
# Checking the result.json DataFrame and uploading it to MinIO

result.to_json('result.json', orient='records')
display(result)
upload_file_to_minio('result.json', 'data', 'data-result/result.json')

In [ ]:
# Reading the result.json from MinIO then remove the duplicated case

path_4 = "s3a://data/data-result/result.json"

df4 = spark.read \
    .json(path_4)
df4.show()

final_result = df4.drop_duplicates(subset=['id'])
final_result.show()